In [7]:
%streaming
%iam_role arn:aws:iam::290620955947:role/LabRole
%region us-east-1
%number_of_workers 2
%idle_timeout 60
%%configure 
{
  "--conf": "spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog",
  "--datalake-formats": "delta"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 1.0.6 and you have 1.0.4 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Previous session type: etl
Setting new session type to Streaming
Current iam_role is None
iam_role has been set to arn:aws:iam::290620955947:role/LabRole.
Previous region: None
Setting new region to: us-east-1
Region is set to: us-east-1
Previous number of workers: None
Setting new number of workers to: 2
Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
The following configurations have been updated: {'--conf': 'spark.sql.extensions=io.delta.sql.Delt

In [1]:
spark

Trying to create a Glue session for the kernel.
Session Type: streaming
Worker Type: G.1X
Number of Workers: 2
Session ID: 40e8e33d-1868-40fd-b1e3-558da6c1da49
Applying the following default arguments:
--glue_kernel_version 1.0.4
--enable-glue-datacatalog true
--conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog
--datalake-formats delta
Waiting for session 40e8e33d-1868-40fd-b1e3-558da6c1da49 to get into ready status...
Session 40e8e33d-1868-40fd-b1e3-558da6c1da49 has been created.


# 1. Environment Configuration

## 1.1 Import dependencies

In [2]:
import boto3
import json
import os
from uuid import uuid4
from datetime import datetime
from datetime import timedelta
from delta.tables import DeltaTable
import time

import pyspark.sql.types as t
import pyspark.sql.functions as f

## 1.2 Constants Variables

In [3]:
BUCKET_NAME = "vproptimiserplatform"
ORDERS = "orders"
STREAM_NAME = "orders_stream"

BRONZE = "bronze"
SILVER = "silver"
GOLD = "gold"
DELTA = "delta"

PROCESSING_TRIGGER = "5 seconds"

## Append Checkpoints 
EVENTS_CHECKPOINT_LOCATION = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/checkpoints/events"
ORDERS_CHECKPOINT_LOCATION = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/checkpoints/orders"
ORDERS_ITEMS_CHECKPOINT_LOCATION = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/checkpoints/orders_items"

## Update Checkpoints
EVENTS_UPDATE_CHECKPOINT_LOCATION = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/checkpoints/events_update"
ORDERS_UPDATE_CHECKPOINT_LOCATION = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/checkpoints/orders_update"
ORDERS_ITEMS_UPDATE_CHECKPOINT_LOCATION = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/checkpoints/orders_items_update"
PRODUCTS_CHECKPOINT_LOCATION = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/checkpoints/inventory_update"

## paths
EVENTS = "events"
ORDERS = "orders"
ORDERS_ITEMS = "orders_items"
PRODUCTS = "products_table"

# TODO TBC medallion architecture??
EVENTS_PATH = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/{BRONZE}/{EVENTS}"
ORDERS_PATH = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/{BRONZE}/{ORDERS}"
ORDERS_ITEMS_PATH = f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/{BRONZE}/{ORDERS_ITEMS}"
PRODUCTS_PATH = f"s3a://{BUCKET_NAME}/{ORDERS}/{GOLD}/{PRODUCTS}"

## Tables
EVENTS_TABLE = f"{EVENTS}_table"
ORDERS_TABLE = f"{ORDERS}_table"
ORDERS_ITEMS_TABLE = f"{ORDERS_ITEMS}_table"

In [4]:
def stop_all_streams(spark):
    """
    Stops all active structured streams in the given Spark session.
    
    Parameters:
    spark (SparkSession): The Spark session with active structured streams.
    
    Returns:
    None
    """
    # Get the list of active streams
    active_streams = spark.streams.active
    
    # Stop each active stream
    for stream in active_streams:
        stream.stop()
    
    # Optionally, print a message confirming that all streams have been stopped
    print(f"Stopped {len(active_streams)} active stream(s).")

In [2]:
stop_all_streams(spark)

NameError: name 'stop_all_streams' is not defined

In [8]:
# def drop_delta_table(PATH):
#     try:
#         # Assuming PATH is your Delta table path
#         deltaTable = DeltaTable.forPath(spark, PATH)
        
#         # Delete the Delta table
#         deltaTable.delete()
        
#         print(f"Delta table at {PATH} successfully deleted.")
        
#     except Exception as e:
#         print(f"Failed to delete Delta table at {PATH}. Error: {str(e)}")
# drop_delta_table(EVENTS_PATH)
# drop_delta_table(ORDERS_PATH)
# drop_delta_table(ORDERS_ITEMS_PATH)

# 2. Orders Stream Subscriber to Kinesis

## 2.1 Read Orders Stream

In [5]:
schema = t.StructType([
    t.StructField("event_id", t.StringType(), True),
    t.StructField("event_type", t.StringType(), True),
    t.StructField("event_timestamp", t.TimestampType(), True),
    t.StructField("order_id", t.StringType(), True),
    t.StructField("order_details", t.StructType([
        t.StructField("customer_id", t.StringType(), True),
        t.StructField("order_date", t.DateType(), True),
        t.StructField("order_timestamp", t.TimestampType(), True),
        t.StructField("items", t.ArrayType(t.StructType([
            t.StructField("product_id", t.StringType(), True),
            t.StructField("product_name", t.StringType(), True),
            t.StructField("price", t.DoubleType(), True),
            t.StructField("weight", t.DoubleType(), True),
            t.StructField("quantity", t.IntegerType(), True),
        ])), True),
        t.StructField("total_amount", t.DoubleType(), True),
        t.StructField("total_weight", t.DoubleType(), True),
        t.StructField("status", t.StringType(), True),
        t.StructField("destination_address", t.StructType([
            t.StructField("address_id", t.StringType(), True),
            t.StructField("neighborhood", t.StringType(), True),
            t.StructField("coordinates", t.ArrayType(t.DoubleType()), True),
            t.StructField("road", t.StringType(), True),
            t.StructField("house_number", t.StringType(), True),
            t.StructField("suburb", t.StringType(), True),
            t.StructField("city_district", t.StringType(), True),
            t.StructField("state", t.StringType(), True),
            t.StructField("postcode", t.StringType(), True),
            t.StructField("country", t.StringType(), True),
            t.StructField("lat", t.DoubleType(), True),
            t.StructField("lon", t.DoubleType(), True)
        ]), True),
        t.StructField("payment_details", t.StructType([
            t.StructField("payment_method", t.StringType(), True),
            t.StructField("payment_status", t.StringType(), True),
            t.StructField("transaction_id", t.StringType(), True)
        ]), True)
    ]), True)
])

> startingPosition indica el punto de incio de lectura del stream, sus opciones son:
>  * trim_horizon: primer dato del stream o el primer registro incluido en el checkpoint
>  * latest: ignora lo previo al ultimo registro

In [30]:
kinesis_order_stream = (
    spark
    .readStream
    .format("kinesis")
    .option("streamName", STREAM_NAME)
    .option("startingPosition", "trim_horizon")
    # .option("startingPosition", "latest")
    .load()
)

In [31]:
kinesis_order_stream.printSchema()

root
 |-- data: binary (nullable = true)
 |-- streamName: string (nullable = true)
 |-- partitionKey: string (nullable = true)
 |-- sequenceNumber: string (nullable = true)
 |-- approximateArrivalTimestamp: timestamp (nullable = true)


In [32]:
df_order_stream = (
    kinesis_order_stream
    .withColumn("json_data", f.col("data").cast("string"))
    .withColumn("orders", f.from_json("json_data", schema))
    .select("orders.*")
)

In [33]:
df_order_stream.printSchema()

root
 |-- event_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_timestamp: timestamp (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_details: struct (nullable = true)
 |    |-- customer_id: string (nullable = true)
 |    |-- order_date: date (nullable = true)
 |    |-- order_timestamp: timestamp (nullable = true)
 |    |-- items: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- product_id: string (nullable = true)
 |    |    |    |-- product_name: string (nullable = true)
 |    |    |    |-- price: double (nullable = true)
 |    |    |    |-- weight: double (nullable = true)
 |    |    |    |-- quantity: integer (nullable = true)
 |    |-- total_amount: double (nullable = true)
 |    |-- total_weight: double (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- destination_address: struct (nullable = true)
 |    |    |-- address_id: string (nullable = true)
 |    |   

## 2.2 Write Orders Streams

### 2.2.1 Events Stream

In [34]:
events_stream = (
    df_order_stream
    .select(
        f.col("event_id"),
        f.col("event_type"),
        f.col("event_timestamp"),
        f.col("order_id")
    )
    .writeStream
    .format("delta")
    .outputMode("append")
    #.trigger(processingTime=PROCESSING_TRIGGER)
    # .trigger(availableNow=True)
    .trigger(once=True)
    .option("path", EVENTS_PATH)
    .option("checkpointLocation", EVENTS_CHECKPOINT_LOCATION)
    # .toTable(EVENTS_TABLE)
    .start()
    )

In [35]:
events_stream.isActive

True


In [36]:
events_stream.status

{'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


In [39]:
events_stream.lastProgress

{'id': 'd7eb6d90-fe48-4b1d-b9c1-87e8ddd048cd', 'runId': '1d0a0810-91a1-4ecd-9d91-b9f8872cb06a', 'name': None, 'timestamp': '2024-07-26T08:41:39.407Z', 'batchId': 8, 'numInputRows': 8, 'inputRowsPerSecond': 0.0, 'processedRowsPerSecond': 0.7508916838746011, 'durationMs': {'addBatch': 9181, 'getBatch': 4, 'getOffset': 205, 'queryPlanning': 13, 'triggerExecution': 10654, 'walCommit': 523}, 'stateOperators': [], 'sources': [{'description': 'KinesisSource[orders_stream]', 'startOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '7'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'endOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '8'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'latestOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '8'}, 's

In [40]:
(
    spark.table(EVENTS_TABLE)
    .orderBy(f.desc("event_timestamp"))
).show()

+--------------------+-----------------+--------------------+--------------------+
|            event_id|       event_type|     event_timestamp|            order_id|
+--------------------+-----------------+--------------------+--------------------+
|ev-5cb27318-fcc9-...|    ORDER_CREATED| 2024-07-26 08:40:45|ord-f88c3b93-4f28...|
|ev-2a502661-c281-...|    ORDER_CREATED| 2024-07-26 08:40:41|ord-f367942e-73a1...|
|ev-7a475bf1-f642-...|    ORDER_CREATED| 2024-07-26 08:40:29|ord-bd2f0dc6-b94c...|
|ev-a3c7688c-b932-...|    ORDER_CREATED| 2024-07-26 08:40:25|ord-b956fd3f-63bd...|
|ev-6652b237-02c0-...|    ORDER_CREATED| 2024-07-26 08:40:21|ord-805aa5c5-631f...|
|ev-539bc1a1-42f6-...|    ORDER_CREATED| 2024-07-26 08:40:13|ord-5cdce260-6424...|
|ev-541e8742-fa99-...|    ORDER_CREATED| 2024-07-26 08:39:57|ord-c3b4a348-f185...|
|ev-7baa2dfb-e269-...|    ORDER_CREATED| 2024-07-26 08:39:46|ord-664677b9-43f7...|
|ev-2625c7ea-1554-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-e5b60f72-19df...|
|ev-

### 2.2.2 Orders Stream

In [41]:
orders_stream = (
    df_order_stream
    .select(
        f.col("order_id"),
        f.col("order_details.customer_id").alias("customer_id"),
        f.col("order_details.total_weight").alias("total_weight"),
        f.col("order_details.total_amount").alias("total_price"),
        f.col("order_details.order_timestamp").alias("order_timestamp"),
        f.col("order_details.status").alias("status"),
        f.col("order_details.destination_address.lat").alias("lat"),
        f.col("order_details.destination_address.lon").alias("lon"),
    )
    .writeStream
    .format("delta")
    .outputMode("append")
    # .trigger(processingTime=PROCESSING_TRIGGER)
    # .trigger(availableNow=True)
    .trigger(once=True)
    .option("path", ORDERS_PATH)
    .option("checkpointLocation", ORDERS_CHECKPOINT_LOCATION)
    # .toTable(ORDERS_TABLE)
    .start()
)

In [42]:
orders_stream.isActive

True


In [43]:
orders_stream.status

{'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


In [44]:
orders_stream.lastProgress

{'id': 'db3c9a58-33fc-4549-a6ed-b258567b9281', 'runId': 'f515e7b6-4bd5-4674-9f7f-bda5cb54b7ab', 'name': None, 'timestamp': '2024-07-26T08:42:14.385Z', 'batchId': 11, 'numInputRows': 8, 'inputRowsPerSecond': 0.0, 'processedRowsPerSecond': 0.9319664492078286, 'durationMs': {'addBatch': 6975, 'getBatch': 3, 'getOffset': 159, 'queryPlanning': 14, 'triggerExecution': 8584, 'walCommit': 573}, 'stateOperators': [], 'sources': [{'description': 'KinesisSource[orders_stream]', 'startOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '10'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'endOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '11'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'latestOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '11'},

In [45]:
(
    spark.table(ORDERS_TABLE)
    .orderBy(f.desc("order_timestamp"))
).show()

+--------------------+--------------------+--------------------+------------------+--------------------+----------+------------------+-------------------+
|            order_id|         customer_id|        total_weight|       total_price|     order_timestamp|    status|               lat|                lon|
+--------------------+--------------------+--------------------+------------------+--------------------+----------+------------------+-------------------+
|ord-f88c3b93-4f28...|cus-35baa105-37fe...|   1.090000033378601|             10.48| 2024-07-26 08:40:45|  RECEIVED|40.436363220214844| -3.770735263824463|
|ord-f367942e-73a1...|cus-293a53e0-2479...|  1.8699999526143074|             76.51| 2024-07-26 08:40:41|  RECEIVED| 40.47103500366211| -3.703655958175659|
|ord-bd2f0dc6-b94c...|cus-685b4364-95bf...|  3.4799998998641968|            104.94| 2024-07-26 08:40:29|  RECEIVED|40.341270446777344|-3.7211179733276367|
|ord-b956fd3f-63bd...|cus-86ddb64c-bbc1...| 0.14000000059604645|      

### 2.2.3 Orders Item stream

In [46]:
df_orders_items_stream = (
    df_order_stream
    .withColumn("exploded_order", f.explode(f.col("order_details.items")))
    .withColumn("inventory_id", f.concat(f.lit("inv-"), f.expr("uuid()")))
    .withColumn("status", f.lit("PENDING"))
    .select(
        f.col("inventory_id"),
        f.col("order_id"),
        f.col("exploded_order.product_id").alias("product_id"),
        f.col("exploded_order.product_name").alias("product_name"),
        f.col("exploded_order.quantity").alias("quantity"),
        f.col("exploded_order.price").alias("price"),
        f.col("exploded_order.weight").alias("weight"),
        f.col("order_details.order_timestamp").alias("order_timestamp"),
        f.col("status")
    )
)

In [47]:
orders_items_stream = (
    df_orders_items_stream
    .writeStream
    .format("delta")
    .outputMode("append")
    #.trigger(processingTime=PROCESSING_TRIGGER)
    #.trigger(availableNow=True)
    .trigger(once=True)
    .option("path", ORDERS_ITEMS_PATH)
    .option("checkpointLocation", ORDERS_ITEMS_CHECKPOINT_LOCATION)
    #.toTable(ORDERS_ITEMS_TABLE)
    .start()
)

In [48]:
orders_items_stream.isActive

True


In [49]:
orders_items_stream.status

{'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


In [51]:
orders_items_stream.lastProgress

{'id': 'e8651aa6-27bb-49c3-9903-a462c6fe4dfd', 'runId': '6058ad20-ed13-4f12-aad0-d694d9a7544f', 'name': None, 'timestamp': '2024-07-26T08:42:51.769Z', 'batchId': 17, 'numInputRows': 8, 'inputRowsPerSecond': 0.0, 'processedRowsPerSecond': 0.9800318510351587, 'durationMs': {'addBatch': 6736, 'getBatch': 4, 'getOffset': 149, 'queryPlanning': 23, 'triggerExecution': 8163, 'walCommit': 537}, 'stateOperators': [], 'sources': [{'description': 'KinesisSource[orders_stream]', 'startOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '16'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'endOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '17'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'latestOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '17'},

In [52]:
(
    spark
    .read
    .format("delta")
    .load(ORDERS_ITEMS_PATH)
    .orderBy(f.desc("order_timestamp"))
).show()

+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+--------------------+----------+
|        inventory_id|            order_id|          product_id|        product_name|quantity| price|              weight|     order_timestamp|    status|
+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+--------------------+----------+
|inv-6887545c-3647...|ord-f88c3b93-4f28...|pro-af3352e0952dd...|Blue Wave 3D Acti...|       1| 10.48|   1.090000033378601| 2024-07-26 08:40:45|   PENDING|
|inv-b3a1fcb1-b429...|ord-f367942e-73a1...|pro-74faf0bdf8fdc...|Star Wars Darth V...|       1| 17.87| 0.27000001072883606| 2024-07-26 08:40:41|   PENDING|
|inv-e3d3f030-08bc...|ord-f367942e-73a1...|pro-3f7200e31bff0...|           Go Matria|       3| 10.22|  0.5199999809265137| 2024-07-26 08:40:41|   PENDING|
|inv-58faf8bc-11f5...|ord-f367942e-73a1...|pro-ed0c348d64037...|Deck B

# 3. Write Inventory Streams

Detailed Workflow Example:

* **Pending**: Customer places an order -> Order is created in the system with status Pending.
* **Confirmed**: Payment is verified -> Status changes to Confirmed.
* **Processing**: Order is being prepared -> Status changes to Processing.
* **Packed**: Items are packed -> Status changes to Packed.
* **Shipped**: Package handed to carrier -> Status changes to Shipped.
* **In Transit**: Carrier updates status -> Status changes to In Transit.
* **Out for Delivery**: Carrier updates status -> Status changes to Out for Delivery.
* **Delivered**: Package delivered to customer -> Status changes to Delivered.

## 3.1 Upsert Product Table

In [54]:
update_products_stream = (
    df_orders_items_stream
    .select(
        f.col("product_id"),
        f.col("quantity")
    )
    .writeStream
    .format("delta")
    .outputMode("update")
    .foreachBatch(upsert_to_products)
    .option("path", PRODUCTS_PATH)
    .option("checkpointLocation", PRODUCTS_CHECKPOINT_LOCATION)
    .trigger(once=True)
    .start()
)

INFO:Callback Server Starting
INFO:Socket listening on ('127.0.0.1', 34723)


In [55]:
update_products_stream.isActive

True


In [56]:
update_products_stream.status

{'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


In [60]:
update_products_stream.lastProgress

{'id': 'd21e3086-feb1-4b29-8fbe-707118f23e3a', 'runId': '547b4c49-0b30-43c8-beb7-870e964bc6fe', 'name': None, 'timestamp': '2024-07-26T08:43:30.988Z', 'batchId': 7, 'numInputRows': 16, 'inputRowsPerSecond': 0.0, 'processedRowsPerSecond': 0.9421185891774126, 'durationMs': {'addBatch': 15543, 'getBatch': 3, 'getOffset': 165, 'queryPlanning': 29, 'triggerExecution': 16983, 'walCommit': 542}, 'stateOperators': [], 'sources': [{'description': 'KinesisSource[orders_stream]', 'startOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '6'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'endOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '7'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'latestOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '7'}, 

In [1]:
(
    spark
    .read
    .format("delta")
    .load(PRODUCTS_PATH)
    .orderBy(f.desc("updated_at"))
    .limit(25)
).show()

NameError: name 'spark' is not defined

## 3.2 Upsert Orders

In [62]:
# (
#     df_order_stream
#       .writeStream
#       .queryName("orders_query")
#       .format("memory")
#       .outputMode("append")
#       .start()
# )

In [63]:
# spark.table("orders_query").show()


+--------------------+-------------+-------------------+--------------------+--------------------+
|            event_id|   event_type|    event_timestamp|            order_id|       order_details|
+--------------------+-------------+-------------------+--------------------+--------------------+
|ev-4bda3bd6-59a5-...|ORDER_CREATED|2024-07-25 08:57:09|ord-93a61b00-1a8d...|{cus-64fc6a25-040...|
|ev-122c578f-5779-...|ORDER_CREATED|2024-07-25 08:57:17|ord-f7b482c0-8ef4...|{cus-05824a8f-ec1...|
|ev-014d2ffc-ed84-...|ORDER_CREATED|2024-07-25 08:57:22|ord-68fe415c-21a4...|{cus-f11ff692-bf2...|
|ev-b6f67afe-456d-...|ORDER_CREATED|2024-07-25 08:57:27|ord-607e1a6e-ca8c...|{cus-e0124b6b-96f...|
|ev-7baa2dfb-e269-...|ORDER_CREATED|2024-07-26 08:39:46|ord-664677b9-43f7...|{cus-80c447f1-7a1...|
|ev-541e8742-fa99-...|ORDER_CREATED|2024-07-26 08:39:57|ord-c3b4a348-f185...|{cus-3d3eafb5-0ff...|
|ev-539bc1a1-42f6-...|ORDER_CREATED|2024-07-26 08:40:13|ord-5cdce260-6424...|{cus-4aa29655-750...|
|ev-6652b2

In [64]:
# (
#     spark.table("orders_query")
#     .withColumn("exploded_order", f.explode(f.col("order_details.items")))
#     .withColumn("inventory_id", f.concat(f.lit("inv-"), f.expr("uuid()")))
#     .withColumn("status", f.lit("PENDING"))
#     .select(
#         f.col("inventory_id"),
#         f.col("order_id"),
#         f.col("exploded_order.product_id").alias("product_id"),
#         f.col("exploded_order.product_name").alias("product_name"),
#         f.col("exploded_order.quantity").alias("quantity"),
#         f.col("exploded_order.price").alias("price"),
#         f.col("exploded_order.weight").alias("weight"),
#         f.col("order_details.order_timestamp").alias("order_timestamp"),
#         f.col("status")
#     )
# ).show()

+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+-------------------+-------+
|        inventory_id|            order_id|          product_id|        product_name|quantity| price|              weight|    order_timestamp| status|
+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+-------------------+-------+
|inv-dc3a8b72-b432...|ord-93a61b00-1a8d...|pro-c5f4c94653a3b...|DC Cover Girls: B...|       2| 84.61|   0.949999988079071|2024-07-25 08:57:09|PENDING|
|inv-51daf8c3-86dc...|ord-93a61b00-1a8d...|pro-5ab08f087af15...|Bif Bang Pow! Her...|       2| 24.99|  0.3400000035762787|2024-07-25 08:57:09|PENDING|
|inv-70894cc9-c17e...|ord-93a61b00-1a8d...|pro-2df8ae3b71b8f...|Bandai Hobby # 4 ...|       2| 30.68| 0.36000001430511475|2024-07-25 08:57:09|PENDING|
|inv-e00be98c-e987...|ord-93a61b00-1a8d...|pro-0aabec43322b0...|VTech Zoo Jamz Pi...|       1|

In [15]:
# (
#     df_orders_items_stream
#       .writeStream
#       .queryName("orders_items_query")
#       .format("memory")
#       .outputMode("append")
#       .start()
# )

In [16]:
# spark.table("orders_items_query").show()

### 3.2.1 Upsert Orders Item Table

In [65]:
def upsert_to_orders_items(microBatchDF, batchId):
    deltaTableOrdersItems = DeltaTable.forPath(spark, ORDERS_ITEMS_PATH)
    (
        deltaTableOrdersItems.alias("t")
        .merge(
            microBatchDF.alias("s"),
            "s.product_id = t.product_id"
        )
        .whenMatchedUpdate(
            set={
                "status": f.lit("PROCESSING"),
                "order_timestamp": f.current_timestamp()
            },
        )
        .execute()
    )

In [66]:
processing_orders_items_stream = (
    df_orders_items_stream
    .filter(f.col("status") == f.lit("PENDING"))
    .select(
        f.col("inventory_id"),
        f.col("product_id")
    )
    .writeStream
    .format("delta")
    .outputMode("update")
    .foreachBatch(upsert_to_orders_items)
    .trigger(once=True)
    .option("path", ORDERS_ITEMS_PATH)
    .option("checkpointLocation", f"{ORDERS_ITEMS_CHECKPOINT_LOCATION}_processing")
    .start()
)


In [67]:
processing_orders_items_stream.isActive

True


In [68]:
processing_orders_items_stream.status

{'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


In [70]:
processing_orders_items_stream.lastProgress

{'id': '15ebff41-8c09-49e9-b3c3-dac2f72e1f3a', 'runId': '5f6b46a9-3eab-44b2-8fd4-f0531607b490', 'name': None, 'timestamp': '2024-07-26T08:45:22.177Z', 'batchId': 1, 'numInputRows': 16, 'inputRowsPerSecond': 0.0, 'processedRowsPerSecond': 1.589351345981921, 'durationMs': {'addBatch': 8693, 'getBatch': 3, 'getOffset': 184, 'queryPlanning': 24, 'triggerExecution': 10067, 'walCommit': 500}, 'stateOperators': [], 'sources': [{'description': 'KinesisSource[orders_stream]', 'startOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '0'}, 'shardId-000000000000': {'iteratorType': 'TRIM_HORIZON', 'iteratorPosition': ''}}, 'endOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '1'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'latestOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '1'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', '

In [73]:
(
    spark
    .read
    .format("delta")
    .load(ORDERS_ITEMS_PATH)
    # .join(
    #     spark.table("orders_items_query"),
    #     "product_id"
    # )
    .orderBy(f.desc("order_timestamp"))
    .limit(25)
).show()

+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+--------------------+----------+
|        inventory_id|            order_id|          product_id|        product_name|quantity| price|              weight|     order_timestamp|    status|
+--------------------+--------------------+--------------------+--------------------+--------+------+--------------------+--------------------+----------+
|inv-648829cc-bdb2...|ord-664677b9-43f7...|pro-ac3367c21c85d...|Vaterra 1/10 1972...|       1|329.99|   5.440000057220459|2024-07-26 08:45:...|PROCESSING|
|inv-67c4403f-9a3e...|ord-664677b9-43f7...|pro-ec71cef1ba89d...|School Smart Viny...|       1|  7.99| 0.18000000715255737|2024-07-26 08:45:...|PROCESSING|
|inv-145058d1-06d8...|ord-664677b9-43f7...|pro-cc5f508a1cc8d...|Ruffneck Scarves ...|       1| 24.37|  0.1899999976158142|2024-07-26 08:45:...|PROCESSING|
|inv-ca7d4930-c7c9...|ord-5cdce260-6424...|pro-38c4cfd60ee22...|VTech 

### 3.2.2 Upsert Orders Table

In [74]:
def upsert_to_orders(microBatchDF, batchId):
    deltaTableOrdersItems = DeltaTable.forPath(spark, ORDERS_PATH)

    microBatchDF = microBatchDF.dropDuplicates(["order_id"])
    (
        deltaTableOrdersItems.alias("t")
        .merge(
            microBatchDF.alias("s"),
            "s.order_id = t.order_id"
        )
        .whenMatchedUpdate(
            set={
                "status": f.lit("PROCESSING"),
                "order_timestamp": f.current_timestamp()
            },
        )
        .execute()
    )

In [75]:
processing_orders_stream = (
    df_orders_items_stream#.dropDuplicates(["order_id"])
    .filter(f.col("status") == f.lit("PENDING"))
    # .withColumn("status", f.lit("PROCESSING"))
    # .withColumn("order_timestamp", f.current_timestamp())
    .select(
        f.col("order_id"),
        # f.col("status"),
        # f.col("order_timestamp")
        # f.col("product_id")
    )
    .writeStream
    .format("delta")
    .outputMode("update")
    .foreachBatch(upsert_to_orders)
    .trigger(once=True)
    .option("path", ORDERS_PATH)
    .option("checkpointLocation", f"{ORDERS_CHECKPOINT_LOCATION}_processing")
    .start()
)

In [76]:
processing_orders_stream.isActive

True


In [77]:
processing_orders_stream.status

{'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


In [78]:
processing_orders_stream.lastProgress

{'id': '7acb5dc3-4988-4b4b-9840-94ac93700591', 'runId': '6094ffbf-e850-4b27-8089-c967f5c0175b', 'name': None, 'timestamp': '2024-07-26T08:47:18.610Z', 'batchId': 2, 'numInputRows': 16, 'inputRowsPerSecond': 0.0, 'processedRowsPerSecond': 1.8761726078799248, 'durationMs': {'addBatch': 6945, 'getBatch': 3, 'getOffset': 207, 'queryPlanning': 17, 'triggerExecution': 8528, 'walCommit': 584}, 'stateOperators': [], 'sources': [{'description': 'KinesisSource[orders_stream]', 'startOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '1'}, 'shardId-000000000000': {'iteratorType': 'TRIM_HORIZON', 'iteratorPosition': ''}}, 'endOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '2'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', 'iteratorPosition': '49654197460728444292469885729742852246329204371326238722'}}, 'latestOffset': {'metadata': {'streamName': 'orders_stream', 'batchId': '2'}, 'shardId-000000000000': {'iteratorType': 'AFTER_SEQUENCE_NUMBER', '

In [79]:
(
    spark
    .read
    .format("delta")
    .load(ORDERS_PATH)
    .orderBy(f.desc("order_timestamp"))
).show()

+--------------------+--------------------+--------------------+------------------+--------------------+----------+------------------+-------------------+
|            order_id|         customer_id|        total_weight|       total_price|     order_timestamp|    status|               lat|                lon|
+--------------------+--------------------+--------------------+------------------+--------------------+----------+------------------+-------------------+
|ord-5cdce260-6424...|cus-4aa29655-750f...|   9.270000010728836|            193.21|2024-07-26 08:47:...|PROCESSING| 40.46931076049805|-3.7590677738189697|
|ord-f367942e-73a1...|cus-293a53e0-2479...|  1.8699999526143074|             76.51|2024-07-26 08:47:...|PROCESSING| 40.47103500366211| -3.703655958175659|
|ord-c3b4a348-f185...|cus-3d3eafb5-0ff4...|  0.4099999852478504|             31.47|2024-07-26 08:47:...|PROCESSING| 40.42833709716797|-3.6563467979431152|
|ord-bd2f0dc6-b94c...|cus-685b4364-95bf...|  3.4799998998641968|      

## 3.3 Append Events

In [80]:
# (
#     spark
#     .readStream
#     .format("delta")
#     .load(ORDERS_PATH)
#     .filter(f.col("status") == f.lit("PROCESSING"))
#     .withColumn(
#         "event_id", f.concat(f.lit("ev-"), f.expr("uuid()"))
#     )
#     .withColumn(
#         "event_type", f.lit("INVENTORY_UPDATED")
#     )
#     .withColumn(
#         "event_timestamp", f.current_timestamp()
#     )
#     .select(
#         f.col("event_id"),
#         f.col("event_type"),
#         f.col("event_timestamp"),
#         f.col("order_id")
#     )
#     .writeStream
#       .queryName("events_query")
#       .format("memory")
#       .outputMode("append")
#       .start()
# )

In [81]:
# spark.table("events_query").show()

+--------------------+-----------------+--------------------+--------------------+
|            event_id|       event_type|     event_timestamp|            order_id|
+--------------------+-----------------+--------------------+--------------------+
|ev-cb947c8f-eb18-...|INVENTORY_UPDATED|2024-07-26 08:49:...|ord-e852ba69-6ecd...|
|ev-fad81b5c-3c39-...|INVENTORY_UPDATED|2024-07-26 08:49:...|ord-ef46b72f-627b...|
|ev-2d9dc609-c4ce-...|INVENTORY_UPDATED|2024-07-26 08:49:...|ord-3a9c47fb-fe10...|
|ev-8293d28d-55d8-...|INVENTORY_UPDATED|2024-07-26 08:49:...|ord-da1fd2fd-840d...|
|ev-a92f0c79-69ba-...|INVENTORY_UPDATED|2024-07-26 08:49:...|ord-e5b60f72-19df...|
|ev-76214ea5-aaf6-...|INVENTORY_UPDATED|2024-07-26 08:49:...|ord-d726c547-c925...|
|ev-6665223c-1463-...|INVENTORY_UPDATED|2024-07-26 08:49:...|ord-3f235839-9683...|
|ev-b9b41c19-4408-...|INVENTORY_UPDATED|2024-07-26 08:49:...|ord-abbd242f-6ef8...|
|ev-1021f19c-14a9-...|INVENTORY_UPDATED|2024-07-26 08:49:...|ord-f7b482c0-8ef4...|
|ev-

In [82]:
processing_events_stream = (
    spark
    .readStream
    .format("delta")
    .load(ORDERS_PATH)
    .filter(f.col("status") == f.lit("PROCESSING"))
    .withColumn(
        "event_id", f.concat(f.lit("ev-"), f.expr("uuid()"))
    )
    .withColumn(
        "event_type", f.lit("INVENTORY_UPDATED")
    )
    .withColumn(
        "event_timestamp", f.current_timestamp()
    )
    .select(
        f.col("event_id"),
        f.col("event_type"),
        f.col("event_timestamp"),
        f.col("order_id")
    )
    .writeStream
    .format("delta")
    .outputMode("append")
    #.trigger(processingTime=PROCESSING_TRIGGER)
    # .trigger(availableNow=True)
    .trigger(once=True)
    .option("path", EVENTS_PATH)
    .option("checkpointLocation", f"{EVENTS_CHECKPOINT_LOCATION}_processing")
    .start()
)

In [83]:
processing_events_stream.isActive

True


In [84]:
processing_events_stream.status

{'message': "Terminated with exception: Detected a data update (for example part-00000-aecccfbb-6659-49e9-a9ce-9445f2513953-c000.snappy.parquet) in the source table at version 16. This is currently not supported. If you'd like to ignore updates, set the option 'ignoreChanges' to 'true'. If you would like the data update to be reflected, please restart this query with a fresh checkpoint directory.", 'isDataAvailable': False, 'isTriggerActive': False}


In [85]:
processing_events_stream.lastProgress

In [86]:
(
    spark
    .read
    .format("delta")
    .load(EVENTS_PATH)
    .limit(15)
).show()

+--------------------+-----------------+--------------------+--------------------+
|            event_id|       event_type|     event_timestamp|            order_id|
+--------------------+-----------------+--------------------+--------------------+
|ev-cce5f790-c6ba-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-e852ba69-6ecd...|
|ev-9dcde3ff-fdea-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-ef46b72f-627b...|
|ev-b96f48cf-941d-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-3a9c47fb-fe10...|
|ev-ef4e8e14-30f2-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-da1fd2fd-840d...|
|ev-2625c7ea-1554-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-e5b60f72-19df...|
|ev-af317f1e-4d59-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-d726c547-c925...|
|ev-7c5297f0-a167-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-3f235839-9683...|
|ev-24e83b2e-1361-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-abbd242f-6ef8...|
|ev-744dd1c0-0a6b-...|INVENTORY_UPDATED|2024-07-26 07:54:...|ord-f7b482c0-8ef4...|
|ev-

# 4. Write Shipping Stream

## 4.1 Get Treshold values

In [87]:
(
    spark
    .read
    .format("delta")
    .load(ORDERS_ITEMS_PATH)
    .filter(f.col("status") == f.lit("PROCESSING"))
    #.groupBy("order_id")
    .agg(
        f.sum("weight").alias("total_weight"),
        f.sum("quantity").alias("total_quantity")
    )
).show()

+-----------------+--------------+
|     total_weight|total_quantity|
+-----------------+--------------+
|55.44000001065433|            72|
+-----------------+--------------+


## 4.2 Route Optimizer model

## 4.3 Append Shippment

## 4.4 Upsert Orders

## 4.5 Append Events

## OLD TESTS 2

In [77]:

# # Define the common condition
# quantity_condition = f.col("products.quantity") >= f.col("orders_items.quantity")


# inventory_stream = (
#     df_orders_items_stream
#     .alias("orders_items")
#     # .withWatermark("orders_items.order_date", "60 seconds")
#     .join(
#         (
#             spark
#             .readStream
#             .format("delta")
#             #.option("readChangeFeed", "true")
#             .load(PRODUCTS_PATH)
#             .alias("products")
#             # .withWatermark("products.updated_at", "150 seconds")
#         ),
#         on=(
#             (f.col("orders_items.product_id") == f.col("products.product_id"))
#             # & (f.col("orders_items.order_date").between(
#             #     f.expr("products.updated_at - interval 30 seconds"),
#             #     f.expr("products.updated_at + interval 30 seconds"))
#             # )
#         ),
#         how="inner"
#     )
#     .withColumn(
#         "quantity_updated",
#         f.when(quantity_condition, f.col("products.quantity") - f.col("orders_items.quantity"))
#          .otherwise(f.col("products.quantity")
#         )
#     )
#     .withColumn(
#         "availability_updated", f.when(quantity_condition, f.lit(True)).otherwise(f.lit(False)
#         )
#     )
#     .withColumn(
#         "status_updated", f.when(quantity_condition, f.lit("PROCESSING")).otherwise(f.lit("INTERRUPTED")
#         )
#     )
#     .withColumn("partitionKey", f.col("inventory_id").cast("string"))
#     .withColumn("data", f.to_json(
#         f.struct(
#             f.col("orders_items.inventory_id").alias("inventory_id"), 
#             f.col("orders_items.order_id").alias("order_id"), 
#             f.col("orders_items.product_id").alias("product_id"),
#             f.col("quantity_updated"),
#             f.col("status_updated"),
#             f.col("availability_updated"),
#         )
#     ))
#     .select("partitionKey", "data")
#     .writeStream
#     .format("kinesis")
#     .option("endpointUrl", "https://kinesis.us-east-1.amazonaws.com")
#     .outputMode("append") 
#     .option("streamName", "inventory_stream")
#     .option("checkpointLocation", ORDERS_INVENTORY_CHECKPOINT_LOCATION)
#     .start()
# )

In [47]:
# # # Define the common condition
# quantity_condition = f.col("products.quantity") >= f.col("orders_items.quantity")

# df_inventory_test = (
#     df_orders_items_stream
#     .alias("orders_items")
#     # .withWatermark("orders_items.order_date", "60 seconds")
#     .join(
#         (
#             spark
#             .readStream
#             .format("delta")
#             #.option("readChangeFeed", "true")
#             .load(PRODUCTS_PATH)
#             .alias("products")
#             # .withWatermark("products.updated_at", "150 seconds")
#         ),
#         on=(
#             (f.col("orders_items.product_id") == f.col("products.product_id"))
#         ),
#         how="inner"
#     )
#     .withColumn(
#         "quantity_updated",
#         f.when(quantity_condition, f.col("products.quantity") - f.col("orders_items.quantity"))
#          .otherwise(f.col("products.quantity")
#         )
#     )
#     .withColumn(
#         "availability_updated", f.when(quantity_condition, f.lit(True)).otherwise(f.lit(False)
#         )
#     )
#     .withColumn(
#         "status_updated", f.when(quantity_condition, f.lit("PROCESSING")).otherwise(f.lit("INTERRUPTED")
#         )
#     )
#     .select(
#         f.col("orders_items.inventory_id").alias("inventory_id"), 
#         f.col("orders_items.order_id").alias("order_id"), 
#         f.col("orders_items.product_id").alias("product_id"), 
#         f.col("quantity_updated"),
#         f.col("status_updated"),
#         f.col("availability_updated"),
#      )
# )

In [ ]:
# (
#     df_orders_items_table
#     .alias("orders_items")
#     .join(
#         df_products.alias("products"), 
#         on=(f.col("orders_items.product_id") == f.col("products.product_id")),
#         how="left"
#     )
#     .withColumn("partitionKey", f.col("inventory_id").cast("string"))
#     .withColumn("data", f.to_json(
#         f.struct(
#             f.col("orders_items.inventory_id").alias("inventory_id"), 
#             f.col("orders_items.order_id").alias("order_id"), 
#             f.col("orders_items.product_id").alias("product_id"), 
#             f.col("orders_items.product_name").alias("product_name"), 
#             f.col("orders_items.quantity").alias("order_quantity"), 
#             f.col("orders_items.price").alias("price"), 
#             f.col("orders_items.weight").alias("weight"),
#             f.col("orders_items.order_date").alias("order_items"),
#             f.col("orders_items.status").alias("status"),
#             f.col("products.quantity").alias("stock_quantity"),
#             f.col("products.availability").alias("availability"),
#         )
#     ))
#     .select("partitionKey", "data")
# ).show()

In [ ]:

# # Define the common condition
# quantity_condition = f.col("products.quantity") >= f.col("orders_items.quantity")


# (
#     df_orders_items_table
#     .alias("orders_items")
#     .join(
#         df_products.alias("products"), 
#         on=(f.col("orders_items.product_id") == f.col("products.product_id")),
#         how="left"
#     )
#     .withColumn(
#         "quantity_updated",
#         f.when(quantity_condition, f.col("products.quantity") - f.col("orders_items.quantity"))
#          .otherwise(f.col("products.quantity")
#         )
#     )
#     .withColumn(
#         "availability_updated",
#         f.when(quantity_condition, f.lit(True))
#          .otherwise(f.lit(False)
#         )
#     )
#     .withColumn(
#         "status_updated",
#         f.when(quantity_condition, f.lit("PROCESSING"))
#          .otherwise(f.lit("INTERRUPTED")
#         )
#     )
#     .select(
#         f.col("orders_items.inventory_id").alias("inventory_id"), 
#         f.col("orders_items.order_id").alias("order_id"), 
#         f.col("orders_items.product_id").alias("product_id"), 
#         #f.col("products.quantity").alias("stock_quantity"),
#         #f.col("orders_items.quantity").alias("order_quantity"),
#         f.col("quantity_updated"),
#         #f.col("orders_items.product_name").alias("product_name"),
#         #f.col("orders_items.price").alias("price"),
#         #f.col("orders_items.weight").alias("weight"),
#         #f.col("orders_items.order_date").alias("order_items"),
#         f.col("status_updated"),
#         f.col("availability_updated"),
#     )
# ).schema

In [ ]:

# # Define the common condition
# quantity_condition = f.col("products.quantity") >= f.col("orders_items.quantity")


# (
#     df_orders_items_table
#     .alias("orders_items")
#     .join(
#         df_products.alias("products"), 
#         on=(f.col("orders_items.product_id") == f.col("products.product_id")),
#         how="left"
#     )
#     .withColumn(
#         "quantity_updated",
#         f.when(quantity_condition, f.col("products.quantity") - f.col("orders_items.quantity"))
#          .otherwise(f.col("products.quantity")
#         )
#     )
#     .withColumn(
#         "availability_updated", f.when(quantity_condition, f.lit(True)).otherwise(f.lit(False)
#         )
#     )
#     .withColumn(
#         "status_updated", f.when(quantity_condition, f.lit("PROCESSING")).otherwise(f.lit("INTERRUPTED")
#         )
#     )
#     .withColumn("partitionKey", f.col("inventory_id").cast("string"))
#     .withColumn("data", f.to_json(
#         f.struct(
#             f.col("orders_items.inventory_id").alias("inventory_id"), 
#             f.col("orders_items.order_id").alias("order_id"), 
#             f.col("orders_items.product_id").alias("product_id"),
#             f.col("quantity_updated"),
#             f.col("status_updated"),
#             f.col("availability_updated"),
#         )
#     ))
#     .select("partitionKey", "data")
# ).show()

In [ ]:
# (
#     df_orders_items_stream
#     .alias("orders_items")
#     .withWatermark("orders_items.order_date", "60 seconds")
#     .join(
#         (
#             spark
#             .readStream
#             .format("delta")
#             .load(PRODUCTS_PATH)
#             .alias("products")
#             .withWatermark("products.updated_at", "150 seconds")
#         ),
#         on=(
#             (f.col("orders_items.product_id") == f.col("products.product_id"))
#             & (f.col("orders_items.order_date").between(
#                 f.expr("products.updated_at - interval 30 seconds"),
#                 f.expr("products.updated_at + interval 30 seconds"))
#             )
#         ),
#         how="left_outer"
#     )
#     .withColumn("partitionKey", f.col("inventory_id").cast("string"))
#     .withColumn("data", f.to_json(
#         f.struct(
#             f.col("orders_items.inventory_id").alias("inventory_id"), 
#             f.col("orders_items.order_id").alias("order_id"), 
#             f.col("orders_items.product_id").alias("product_id"), 
#             f.col("orders_items.product_name").alias("product_name"), 
#             f.col("orders_items.quantity").alias("order_quantity"), 
#             f.col("orders_items.price").alias("price"), 
#             f.col("orders_items.weight").alias("weight"),
#             f.col("orders_items.order_date").alias("order_items"),
#             f.col("orders_items.status").alias("status"),
#             f.col("products.quantity").alias("order_quantity"),
#             f.col("products.availability").alias("availability"),
#         )
#     ))
# )

In [ ]:
# inventory_stream = (
#    df_orders_items_stream
#    .withColumn("partitionKey", f.col("inventory_id").cast("string"))
#    .writeStream
#    .format("kinesis")
#    .option("endpointUrl", "https://kinesis.us-east-1.amazonaws.com")
#    .outputMode("append")
#    .trigger(processingTime=PROCESSING_TRIGGER)
#    .option("streamName", "inventory_stream")
#    .option("checkpointLocation", f"s3://{BUCKET_NAME}/checkpoints/messages-user-stream")
#    .option("partitionKey", "partitionKey")
#    .start()
#)

## OLD TESTS 1

In [ ]:
# ## TBC: trigger integration for watermark
# inventory_stream = (
#     df_orders_items_stream
#     .withColumn("partitionKey", f.col("inventory_id").cast("string"))
#     .withColumn("data", f.to_json(
#         f.struct(
#             f.col("inventory_id"), 
#             f.col("order_id"), 
#             f.col("product_id"), 
#             f.col("product_name"), 
#             f.col("quantity"), 
#             f.col("price"), 
#             f.col("weight"),
#             f.col("order_date"),
#             f.col("status")
#         )
#     ))
#     .select("partitionKey", "data")
#     .writeStream
#     .format("kinesis")
#     .option("endpointUrl", "https://kinesis.us-east-1.amazonaws.com")
#     .outputMode("append") 
#     .option("streamName", "inventory_stream")
#     .option("checkpointLocation", f"s3://{BUCKET_NAME}/checkpoints/inventory-stream")
#     .start()
# )

In [ ]:
# def upsert_to_inventory(microBatchOutputDF, batchId):

#     deltaTable = DeltaTable.forPath(spark, PRODUCTS_PATH)
    
#     (deltaTable.alias("t")
#         .merge(
#             microBatchOutputDF.alias("s"),
#             "s.product_id = t.product_id"
#         )
#         .whenMatchedUpdate(set={
#             # condition="t.quantity" > s.quantity"
#             "quantity": f.col("t.quantity") - f.col("s.quantity"),  # Direct string assignment
#         })
#         .execute()
#     )

In [ ]:
# def upsert_to_orders(microBatchOutputDF, batchId):

#     deltaTable = DeltaTable.forPath(spark, ORDERS_PATH)
    
#     (deltaTable.alias("t")
#         .merge(
#             microBatchOutputDF.alias("s"),
#             "s.order_id = t.order_id"
#         )
#         .whenMatchedUpdate(set={
#             # condition="t.status" != VAL '5' DAY)"
#             "status": f.when(f.col("s.items") > f.col("t.items"), f.lit("PROCESSING"))
#                         .otherwise(f.lit("CANCELLED")),  # Direct string assignment
#             "order_date": f.current_timestamp() + f.expr("INTERVAL 5 minutes")  # Use expr for date calculation
#         })
#         .execute()
#     )

In [ ]:
# update_orders_stream = (
#     df_order_stream
#     .select(
#         f.col("order_id"),
#         f.col("order_details.customer_id").alias("customer_id"),
#         f.col("order_details.total_weight").alias("total_weight"),
#         f.col("order_details.total_amount").alias("total_price"),
#         f.col("order_details.order_date").alias("order_date"),
#         f.col("order_details.status").alias("status"),
#         f.col("order_details.destination_address.lat").alias("lat"),
#         f.col("order_details.destination_address.lon").alias("lon"),
#     )
#     .writeStream
#     .format("delta")
#     .foreachBatch(upsert_to_orders)
#     .outputMode("update")
#     .option("path", ORDERS_PATH)
#     .option("checkpointLocation", f"s3a://{BUCKET_NAME}/{ORDERS}/{DELTA}/checkpoints/orders_update")
#     .trigger(processingTime=PROCESSING_TRIGGER)
#     .start()    
# )

In [ ]:
# (
#     spark.table(ORDERS_TABLE)
#     .select(f.col("status")).distinct()
#     .show()
# )

In [ ]:
# (
#     spark
#     .read
#     .format("delta")
#     .load(ORDERS_PATH)
#     .filter(f.col("status") != "PROCESSING")
#     .count()
# )

In [ ]:
# # weight treshold 
# (
#     df_orders_bronze
#     .select(
#         f.sum(f.col("order_details.total_weight")).alias("total_weight")
#     ).collect()[0]["total_weight"]
# )